In [ ]:
!python --version
%pip install jax jaxlib --upgrade

In [ ]:
import jax.numpy as jnp
from scipy import signal
import numpy as np
import time

from jaxdsp import config, training
from jaxdsp.processors import fir_filter, iir_filter, clip, delay_line, lowpass_feedback_comb_filter as lbcf, allpass_filter, freeverb, sine_wave, serial_processors
from jaxdsp.plotting import plot_filter, plot_loss, plot_params, plot_optimization
import jaxdsp.loss

In [ ]:
def array_with_one_at(i, size):
    ar = np.zeros(size)
    ar[i] = 1.0
    return ar

In [ ]:
step_size = 1.0
num_batches = 100
num_train = 100
Xs_random = np.random.randn(num_train, config.buffer_size)
Xs_unit = np.array([array_with_one_at(i, config.buffer_size) for i in range(num_train)])
Xs_chirp = np.array(np.split(signal.chirp(np.linspace(0, 10, num_train * config.buffer_size), f0=300, f1=1, t1=10), num_train))
#Xs_chirp = np.ones((n_train,1)) * signal.chirp(np.linspace(0, 10, n_samples), f0=10, f1=1, t1=10)

In [ ]:
def evaluate_processors(processor, processor_config=None, loss_fn=jaxdsp.loss.mse, Xs=Xs_chirp, reference_fn=None,
                        plot_loss_history=True, plot_params_history=True, title=None):
    processor_config = processor_config or processor.config()
    carry_target = {'params': processor_config.params_target, 'state': processor_config.state_init}
    trainer = training.IterativeTrainer(processor, training.Config(loss_fn=loss_fn, step_size=step_size),
                                        processor_config, track_history=True)
    start = time.time()
    for i in range(num_batches):
        X = Xs[i % Xs.shape[0]]
        carry_target, Y_target = processor.tick_buffer(carry_target, X)
        trainer.step(X, Y_target)

    params_estimated = trainer.params()
    carry_estimated = {'params': params_estimated, 'state': trainer.processor_state}
    params_target = carry_target['params']

    print('Train time: {:.3E} s'.format(time.time() - start))
    print('Loss: {:.3E}'.format(trainer.loss))
    print('Estimated params: ', params_estimated)

    X_eval = Xs[0]
    Y_estimated, Y_target = training.evaluate(carry_estimated, carry_target, processor, X_eval)
    Y_reference = reference_fn(X_eval, params_target) if reference_fn is not None else None

    if plot_loss_history:
        plot_loss(trainer.step_evaluator.loss_history)
    if plot_params_history:
        plot_params(params_target, trainer.step_evaluator.params_history)
    plot_filter(X_eval, Y_target, Y_reference, Y_estimated, title)

In [ ]:
evaluate_processors(sine_wave, loss_fn=jaxdsp.loss.spectral)

In [ ]:
# evaluate_processors([freeverb])

In [ ]:
evaluate_processors(serial_processors, serial_processors.config([lbcf]), title=lbcf.NAME)

In [ ]:
evaluate_processors(serial_processors, serial_processors.config([lbcf]), title=lbcf.NAME)

In [ ]:
evaluate_processors([allpass_filter])

In [ ]:
evaluate_processors([delay_line])

In [ ]:
evaluate_processors([fir_filter], reference_fn=lambda X, params: signal.lfilter(params[fir_filter.NAME]['B'], [1.0], X))

In [ ]:
evaluate_processors([iir_filter], reference_fn=lambda X, params: signal.lfilter(params[iir_filter.NAME]['B'], params[iir_filter.NAME]['A'], X), plot_params_history=False)

In [ ]:
evaluate_processors([clip], reference_fn=lambda X, params: np.clip(X, params[clip.NAME]['min'], params[clip.NAME]['max']))

In [ ]:
evaluate_processors([iir_filter, clip])

## Non-batch training example

In [ ]:
from jaxdsp import training
from jax import jit, lax

processor = serial_processors
processor_config = processor.config([lbcf])
Xs = Xs_chirp
carry_target = {'params': processor_config.params_target, 'state': processor_config.state_init}
trainer = training.IterativeTrainer(processor, training.Config(step_size=0.1), processor_config, track_history=True)
for i in range(200):
    X = Xs[i % Xs.shape[0]]
    carry_target, Y_target = processor.tick_buffer(carry_target, X)
    trainer.step(X, Y_target)

trainer.params_and_loss()

In [ ]:
param_inits = [{'frequency_hz': freq, 'phase': 0.0} for freq in np.linspace(100, 140, 50)]
params_target = {'frequency_hz': 120.0, 'phase': 0.0}
plot_optimization(sine_wave, Xs_chirp, param_inits, params_target, 'frequency_hz', 10)

In [ ]:
# TODO: verify Lagrange interpolation converges over a 4-sample range

## Implementing Freeverb

From [Physical Audio Signal Processing](https://ccrma.stanford.edu/~jos/pasp/Freeverb.html):

![](https://ccrma.stanford.edu/~jos/pasp/img728_2x.png)

It is composed of [lowpass feedback comb filters](https://ccrma.stanford.edu/~jos/pasp/Lowpass_Feedback_Comb_Filter.html) and [Schroeder allpass sections](https://ccrma.stanford.edu/~jos/pasp/Schroeder_Allpass_Sections.html).

In order to implement Freeverb in a differentiable way, I will use plain IIR filters for each component. (Note that this is extremely inefficient compared with the direct implementation of the difference equation.) (See issues in differentiable parameterizing of delay line length above.)


$H_{AP}(z)= \dfrac{X(z)}{Y(z)} =\dfrac{g^∗+z^{−m}}{1+gz^{−m}}$

$\begin{align}
H_{LBCF}(z) &= \dfrac{z^{-N}}{1-f\frac{1-d}{1-dz^{-1}}z^{-N}}\\
&= \dfrac{z^{-N}}{\frac{1-dz^{-1}-f(1-d)z^{-N}}{1-dz^{-1}}}\\
&= \dfrac{z^{-N}(1-dz^{-1})}{1-dz^{-1}-f(1-d)z^{-N}}\\
&= \dfrac{-dz^{-N-1}+z^{-N}}{1-dz^{-1}-f(1-d)z^{-N}}\\
\end{align}$

TODO: verify the behavior is identical

In [ ]:
# http://devmaster.net/forums/topic/4648-fast-and-accurate-sinecosine/
def sine_approx(x):
    y = 4/np.pi * x - 4/(np.pi**2) * x * np.abs(x)
    return y * (0.225 * (np.abs(y) - 1) + 1)

In [ ]:
from matplotlib import pyplot as plt

x = np.linspace(0, 20, 100) % (2 * np.pi) - np.pi
plt.plot(sine_approx(x))
plt.plot(np.sin(x))

In [ ]:
plt.plot(np.abs(jnp.fft.rfft(jnp.sin(jnp.linspace(-np.pi, np.pi, 50, True)))) / 50)

In [ ]:
from jax.tree_util import tree_map, tree_multimap

params_init = {'hi': 1.0, 'bye': {'dry': 4}}
params_history = tree_map(lambda param: [param], params_init)
new_params = {'hi': 2.0, 'bye': {'dry': 5}}
params_history = tree_multimap(lambda h, param: param + [h], new_params, params_history)
new_params = {'hi': 3.0, 'bye': {'dry': 6}}
params_history = tree_multimap(lambda h, param: param + [h], new_params, params_history)

In [ ]:
params_history